In [5]:
using Pkg
using Revise
Pkg.activate(dirname(dirname(@__DIR__)))
using Macro
using Gurobi
using Plots
using DataFrames, CSV
using BenchmarkTools
using JuMP

  Activating project at `~/Macro`


In [27]:
case_path = @__DIR__
println("###### ###### ######")
println("Running case at $(case_path)")

system = Macro.load_system(case_path)

model = Macro.generate_model(system)

Macro.set_optimizer(model, Gurobi.Optimizer);

Macro.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)

Macro.optimize!(model)

###### ###### ######
Running case at /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement_test


┌ Info: Loading JSON data from /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement_test/system_data.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement_test/system_data.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from system/nodes.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading CSV data from system/demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:12
┌ Info: Loading CSV data from system/demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:14
┌ Info: Loading CSV data from system/hourly_cement_demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:12
┌ Info: Loading CSV data from system/hourly_cement_demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:14
┌ Info: Loading JSON data from assets/

system.settings.Scaling = false


┌ Info: Starting model generation
└ @ Macro /home/al3792/Macro/src/generate_model.jl:3
┌ Info: Adding linking variables
└ @ Macro /home/al3792/Macro/src/generate_model.jl:17
┌ Info: Defining available capacity
└ @ Macro /home/al3792/Macro/src/generate_model.jl:20
┌ Info: Generating planning model
└ @ Macro /home/al3792/Macro/src/generate_model.jl:23
┌ Info: Generating operational model
└ @ Macro /home/al3792/Macro/src/generate_model.jl:26
┌ Info: Model generation complete, it took 14.008322954177856 seconds
└ @ Macro /home/al3792/Macro/src/generate_model.jl:31


Set parameter LicenseID to value 197246
Set parameter GURO_PAR_SPECIAL
Set parameter TokenServer to value "license.rc.princeton.edu"
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Red Hat Enterprise Linux 8.10 (Ootpa)")

CPU model: Intel(R) Xeon(R) Gold 6246R CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 744619 rows, 481835 columns and 2521829 nonzeros
Model fingerprint: 0x75e58f85
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+04]
Presolve removed 245858 rows and 140737 columns
Presolve time: 1.00s
Presolved: 498761 rows, 341098 columns, 1811942 

In [28]:
case = "capacity_no_co2_cap"

capacity_results = Macro.get_optimal_asset_capacity(system)
results_dir = joinpath(case_path, "results")
mkpath(results_dir)
Macro.write_csv(joinpath(results_dir, case * "_capacity.csv"), capacity_results)
println("wrote results")

wrote results


In [29]:
co2_nodes = Macro.get_nodes_sametype(system.locations, CO2)

1-element Vector{Node}:
 Node{CO2}(:co2_sink, Macro.TimeData{CO2}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0)), Dict(:exogenous => Dict()), AbstractTypeConstraint[], Dict{Any, Any}(:exogenous => 1-dimensional DenseAxisArray{AffExpr,1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{AffExpr}:
 vFLOW_cement_MA_co2_edge[1] + vFLOW_cement_CT_co2_edge[1] + vFLOW_ng_MA_co2_edge[1] + vFLOW_ng_CT_co2_edge[1] + vFLOW_ng_ME_co2_edge[1]
 vFLOW_cement_MA_co2_edge[2] + vFLOW_cement_CT_co2_edge[2] + vFLOW_ng_MA_co2_edge[2] + vFLOW_ng_CT_co2_edge[2] + vFLOW_ng_ME_co2_edge[2]
 vFLOW_cement_MA_co2_edge[3] + vFLOW_cement_CT_co2_edge[3] + vFLOW_ng_MA_co2_edge[3] + vFLOW_ng_CT_co2_edge[3] + vFLOW_ng_ME_co2_edge[3]
 vFLOW_cement_MA_co2_edge[4] + vFLOW_cement_CT_co2_edge[4] + vFLOW_ng_MA_co2_edge[4] + vFLOW_ng_CT_co2_edge[4] + vFLOW_ng_ME_co2_edge[4]
 vFLOW_cement_MA_co2_edge[5] + vFLOW_cement_CT_co2_edge[5] + vFLOW_ng_MA_co2_edge[5] + vFLOW_ng_CT_co2_e

In [31]:
results_8760 = DataFrame(
    cement_plant_1 = Macro.value.(Macro.flow(system.assets[1].cement_edge)).data,
    cement_plant_2 = Macro.value.(Macro.flow(system.assets[2].cement_edge)).data,
    #cement_plant_3 = Macro.value.(Macro.flow(system.assets[3].cement_edge)).data,

    cement_plant_1_emissions = Macro.value.(Macro.flow(system.assets[1].co2_edge)).data,
    cement_plant_2_emissions = Macro.value.(Macro.flow(system.assets[2].co2_edge)).data,
    #cement_plant_3_emissions = Macro.value.(Macro.flow(system.assets[3].co2_edge)).data,

    ng_1_emissions = Macro.value.(Macro.flow(system.assets[8].co2_edge)).data,
    ng_2_emissions = Macro.value.(Macro.flow(system.assets[9].co2_edge)).data,
    ng_3_emissions = Macro.value.(Macro.flow(system.assets[10].co2_edge)).data,

    total_emissions = Macro.value.(co2_nodes[1].operation_expr[:exogenous].data)
)

Row,cement_plant_1,cement_plant_2,cement_plant_1_emissions,cement_plant_2_emissions,ng_1_emissions,ng_2_emissions,ng_3_emissions,total_emissions
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,17.5661,82.4339,13.6313,63.9687,1823.62,197.566,2.2792,2101.06
2,17.566,82.434,13.6312,63.9688,1713.97,197.548,2.29079,1991.41
3,17.566,82.434,13.6312,63.9688,1632.43,197.467,2.29388,1909.79
4,17.566,82.434,13.6312,63.9688,1591.41,197.401,2.29007,1868.71
5,17.5659,82.4341,13.6312,63.9688,1585.01,197.471,2.28015,1862.36
6,17.5653,82.4347,13.6307,63.9693,1616.46,197.497,2.25853,1893.81
7,17.5653,82.4347,13.6307,63.9693,1684.05,197.427,2.2276,1961.3
8,17.5659,82.4341,13.6311,63.9689,1745.35,197.193,2.18864,2022.33
9,17.5662,82.4338,13.6314,63.9686,1848.91,197.048,2.2594,2125.81


In [32]:
maximum(eachrow(results_8760[!, 1:2]))

Row,cement_plant_1,cement_plant_2
,Float64,Float64
6727,17.569,82.431


In [18]:
system.assets[1].cement_edge.constraints

1-element Vector{AbstractTypeConstraint}:
 CapacityConstraint(missing, missing, 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 -vNEWCAP_cement_MA_cement_edge + vRETCAP_cement_MA_cement_edge + vFLOW_cement_MA_cement_edge[1] ≤ 0
 -vNEWCAP_cement_MA_cement_edge + vRETCAP_cement_MA_cement_edge + vFLOW_cement_MA_cement_edge[2] ≤ 0
 -vNEWCAP_cement_MA_cement_edge + vRETCAP_cement_MA_cement_edge + vFLOW_cement_MA_cement_edge[3] ≤ 0
 -vNEWCAP_cement_MA_cement_edge + vRETCAP_cement_MA_cement_edge + vFLOW_cement_MA_cement_edge[4] ≤ 0
 -vNEWCAP_cement_MA_cement_edge + vRETCAP_cement_MA_cement_edge + vFLOW_cement_MA_cement_edge[5] ≤ 